Compare solar panel/farm data from REPD, FiT and OSM
====

Notes:
1. Make project Onedrive and put all data on there
2. Put original data files into a /raw
3. Separate dir for the manually edited e.g. fit csv below (could use symbolic link)
4. Make a document of all the fields in each dataset
4. What set of columns identify a row (especially for fit)?
5. Precisely what is in OSM for REPD tagged objects (click on them)
6. Generate postres database from csvs via a script

FIND 5 EXAMPLES:
- Table of fields
- Include all OSM fields and relevant REPD ones

In [1]:
import pandas as pd
import numpy as np
import statistics as st

In [152]:
# FiT data in a csv concatenation of all three spreadsheets, missing from repo as file too large
repd = "../data/REPD/Public-Database-September-2019_edited.csv"
fit = "../data/FiT/feed-in_tariff_installation_report_30_september_2019.csv"
osm = "../data/open-street-maps/solarpv-osm-uk-data-20191117/dan_stowell_osm_analysis/compile_processed_PV_objects.csv"

In [153]:
repd_df = pd.read_csv(repd, encoding = "ISO-8859-1")

In [154]:
repd_df.head()

,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,RO Banding (ROC/MWh),FiT Tariff (p/kWh),CfD Capacity (MW),Turbine Capacity (MW),No. of Turbines,Height of Turbines (m),Mounting Type for Solar,Development Status,Development Status (short),Address,County,Region,Country,Post Code,X-coordinate,Y-coordinate,Planning Authority,Planning Application Reference,Appeal Reference,Secretary of State Reference,Type of Secretary of State Intervention,Judicial Review,Offshore Wind Round,Planning Application Submitted,Planning Application Withdrawn,Planning Permission Refused,Appeal Lodged,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational
0,N00053B,1,07/07/2009,RWE npower,Aberthaw Power Station Biomass,Biomass (co-firing),NaN,NaN,35,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Operational,Operational,"Aberthaw Power Station\r, The Leys\r, Aberthaw...",South Glamorgan,Wales,Wales,CF62 4ZW,"302,219","166,320",Vale of Glamorgan Council,04/00580/FUL,NaN,NaN,NaN,NaN,NaN,16/04/2004,NaN,NaN,NaN,NaN,NaN,NaN,03/09/2004,NaN,NaN,NaN,NaN,01/05/2006,01/05/2007
1,AA110,2,20/11/2017,Orsted (formerly Dong Energy) / Peel Energy,Hunterston - cofiring,Biomass (co-firing),NaN,NaN,170,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Planning Application Withdrawn,Application Withdrawn,"Hunterston, Fairlie, West Kilbride, Ayrshire",Strathclyde,Scotland,Scotland,NaN,"218,740","652,937",Scottish Government (S36),NaN,NaN,NaN,NaN,NaN,NaN,02/06/2010,27/06/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B0730,3,23/04/2019,Scottish and Southern Energy (SSE),Ferrybridge Multifuel 2 (FM2),EfW Incineration,NaN,NaN,70,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Under Construction,Under Construction,NaN,West Yorkshire,Yorkshire and Humber,England,NaN,"447,490","424,684",The Planning Inspectorate - National Infrastru...,NaN,NaN,NaN,NaN,NaN,NaN,20/08/2014,NaN,NaN,NaN,NaN,NaN,NaN,28/10/2015,NaN,NaN,NaN,28/10/2020,01/09/2016,NaN
3,1106000,4,18/12/2003,Energy Power Resources,Thetford Biomass Power Station,Biomass (dedicated),NaN,NaN,38.5,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Operational,Operational,"Thetford Power Station\r, Marley Building Mate...",Norfolk,Eastern,England,NaN,"585,300","286,900",Breckland Council,NaN,NaN,NaN,NaN,NaN,NaN,23/06/1994,NaN,NaN,NaN,NaN,NaN,NaN,05/05/1995,NaN,NaN,NaN,NaN,NaN,02/10/1998
4,2047000,5,29/09/2005,Agrigen,Nunn Mills Road Biomass Plant,Biomass (dedicated),NaN,NaN,8.8,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Planning Permission Refused,Application Refused,"Nunn Mills Road\r, Northampton\r, Northamptons...",Northamptonshire,East Midlands,England,NaN,"476,500","259,700",Northamptonshire County Council,NaN,NaN,NaN,NaN,NaN,NaN,24/06/1997,NaN,01/02/1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
fit_df = pd.read_csv(fit, encoding = "ISO-8859-1")

In [156]:
fit_df.head()

,Extension (Y/N),Postcode,Technology,Installed capacity,Declared net capacity,Application date,Commissioning date,MCS issue date,Export status,Tariff code,Tariff description,Installation type,Country,Local authority,Government office region,Constituency,Accreditation Route,MPAN Prefix,Community/school category,LLSOA code
0,N,FK2,Photovoltaic,0.83,0.83,04/07/2019 00:00,14/06/2018 00:00,26/06/2018 11:19,Export (Deemed),PV-N/0-4/09H-1,PV (<=4kW (new build))-2018/19 - H,Domestic,Scotland,Falkirk,Scotland,Falkirk,MCS,24.0,Not Community Or School,Unknown
1,N,LE4,Photovoltaic,18.63,18.63,04/01/2019 00:00,30/11/2018 00:00,12/12/2018 13:24,Export (Deemed),PV/10-50/09L-3,PV (>10-<=50kW)-2018/19 - L,Non Domestic (Commercial),England,Leicester,East Midlands,Leicester West,MCS,11.0,Not Community Or School,E01013680
2,N,GU22,Photovoltaic,1.38,1.00,16/05/2019 00:00,20/06/2017 00:00,20/06/2017 16:37,Export (Deemed),PV-N/0-4/08L-1,PV (<=4kW (new build))-2017/18 - L,Domestic,England,Woking,South East,Woking,MCS,27.0,Not Community Or School,Unknown
3,N,BS34,Photovoltaic,5.30,3.00,21/01/2019 00:00,12/12/2016 00:00,17/12/2018 15:13,Export (Deemed),PV/4-10/09H-3,PV (4-10kW)-2018/19 - H,Domestic,England,South Gloucestershire,South West,Filton and Bradley Stoke,MCS,22.0,Not Community Or School,Unknown
4,N,HU15,Photovoltaic,3.92,3.60,26/03/2019 00:00,18/01/2019 00:00,30/01/2019 11:15,Export (Deemed),PV-R/0-4/09L-4,PV (<=4kW (retrofit))-2018/19 - L,Domestic,England,East Riding of Yorkshire,Yorkshire and The Humber,Haltemprice and Howden,MCS,23.0,Not Community Or School,E01012968


In [157]:
osm_df = pd.read_csv(osm, encoding = "ISO-8859-1")

In [158]:
osm_df.head()

,objtype,id,user,timestamp,lat,lon,calc_area,calc_capacity,generator:solar:modules,location,orientation,plantref,tag_power,tag_repd:id,tag_start_date
0,node,383729355,Dan023,2019-11-07T15:49:12Z,53.045913,-1.410764,0.0,NaN,NaN,roof,NaN,NaN,generator,NaN,NaN
1,node,432708661,don-vip,2011-06-03T16:19:07Z,57.148783,-6.211683,0.0,NaN,NaN,NaN,NaN,NaN,generator,NaN,NaN
2,node,754144142,Gregory Williams,2019-09-07T05:49:05Z,51.421184,0.824588,0.0,NaN,NaN,roof,NaN,NaN,generator,NaN,NaN
3,node,914501375,doublah,2019-08-09T16:21:10Z,51.504658,-0.104954,0.0,15.0,NaN,NaN,NaN,NaN,generator,NaN,2008
4,node,916163617,ehm1806,2012-12-14T07:16:28Z,51.464968,-0.073252,0.0,NaN,NaN,NaN,NaN,NaN,generator,NaN,NaN


In [159]:
len(osm_df)

126939

How many of the REPD entries are already in OSM?
====

Check the ones already tagged all there.

In [160]:
len(repd_df)

5686

In [161]:
osm_df['tag_repd:id'].notnull().sum()

893

Find REPD entries not already in OSM?
====

Can we use coordinates?

In [162]:
from bng_to_latlon import OSGB36toWGS84 as convert

In [163]:
convert(538890, 177320)

(51.47779538331092, -0.0014016837826672265)

In [164]:
# Convert each BNG coordinate to lat and lon
for index, row in repd_df.iterrows():
    # Ignore any rows that don't have an X and Y coordinate
    if type(row['X-coordinate']) == str and type(row['Y-coordinate']) == str:
        x = int(row['X-coordinate'].replace(',', ''))
        y = int(row['Y-coordinate'].replace(',', ''))
    lat, lon = convert(x, y)
    repd_df.set_value(index,'latitude', lat)
    repd_df.set_value(index,'longitude', lon)
repd_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,RO Banding (ROC/MWh),FiT Tariff (p/kWh),CfD Capacity (MW),Turbine Capacity (MW),No. of Turbines,Height of Turbines (m),Mounting Type for Solar,Development Status,Development Status (short),Address,County,Region,Country,Post Code,X-coordinate,Y-coordinate,Planning Authority,Planning Application Reference,Appeal Reference,Secretary of State Reference,Type of Secretary of State Intervention,Judicial Review,Offshore Wind Round,Planning Application Submitted,Planning Application Withdrawn,Planning Permission Refused,Appeal Lodged,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational,latitude,longitude
0,N00053B,1,07/07/2009,RWE npower,Aberthaw Power Station Biomass,Biomass (co-firing),NaN,NaN,35,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Operational,Operational,"Aberthaw Power Station\r, The Leys\r, Aberthaw...",South Glamorgan,Wales,Wales,CF62 4ZW,"302,219","166,320",Vale of Glamorgan Council,04/00580/FUL,NaN,NaN,NaN,NaN,NaN,16/04/2004,NaN,NaN,NaN,NaN,NaN,NaN,03/09/2004,NaN,NaN,NaN,NaN,01/05/2006,01/05/2007,51.387471,-3.406610
1,AA110,2,20/11/2017,Orsted (formerly Dong Energy) / Peel Energy,Hunterston - cofiring,Biomass (co-firing),NaN,NaN,170,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Planning Application Withdrawn,Application Withdrawn,"Hunterston, Fairlie, West Kilbride, Ayrshire",Strathclyde,Scotland,Scotland,NaN,"218,740","652,937",Scottish Government (S36),NaN,NaN,NaN,NaN,NaN,NaN,02/06/2010,27/06/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.735807,-4.888574
2,B0730,3,23/04/2019,Scottish and Southern Energy (SSE),Ferrybridge Multifuel 2 (FM2),EfW Incineration,NaN,NaN,70,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Under Construction,Under Construction,NaN,West Yorkshire,Yorkshire and Humber,England,NaN,"447,490","424,684",The Planning Inspectorate - National Infrastru...,NaN,NaN,NaN,NaN,NaN,NaN,20/08/2014,NaN,NaN,NaN,NaN,NaN,NaN,28/10/2015,NaN,NaN,NaN,28/10/2020,01/09/2016,NaN,53.716417,-1.281885
3,1106000,4,18/12/2003,Energy Power Resources,Thetford Biomass Power Station,Biomass (dedicated),NaN,NaN,38.5,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Operational,Operational,"Thetford Power Station\r, Marley Building Mate...",Norfolk,Eastern,England,NaN,"585,300","286,900",Breckland Council,NaN,NaN,NaN,NaN,NaN,NaN,23/06/1994,NaN,NaN,NaN,NaN,NaN,NaN,05/05/1995,NaN,NaN,NaN,NaN,NaN,02/10/1998,52.448628,0.725273
4,2047000,5,29/09/2005,Agrigen,Nunn Mills Road Biomass Plant,Biomass (dedicated),NaN,NaN,8.8,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Planning Permission Refused,Application Refused,"Nunn Mills Road\r, Northampton\r, Northamptons...",Northamptonshire,East Midlands,England,NaN,"476,500","259,700",Northamptonshire County Council,NaN,NaN,NaN,NaN,NaN,NaN,24/06/1997,NaN,01/02/1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.230195,-0.881341


Compare coordinates for REPD entries in OSM that we know are already there
----

In [165]:
def get_float(x):
    if type(x) == str:
        return float(x.split(';')[0])  # TODO: this ignores all but 1st tagged repd id
    return x
osm_df['tag_repd:id'] = osm_df['tag_repd:id'].apply(lambda x: get_float(x))

In [166]:
osm_df['tag_repd:id'].notnull().sum()

893

In [167]:
repd_osm_merge_df = pd.merge(repd_df, osm_df, left_on='Ref ID', right_on='tag_repd:id')

In [168]:
# should be the same as the number of tag_repd:id in osm_df
len(repd_osm_merge_df)

892

Note: How far off are the outliers?
- Get the med, max, min, upper and lower quartile (5 num summary)

In [216]:
mean_lon_offset = st.mean(repd_osm_merge_df['lon'] - repd_osm_merge_df['longitude'])
max_lon_offset = max(repd_osm_merge_df['lon'] - repd_osm_merge_df['longitude'])
mean_lat_offset = st.mean(repd_osm_merge_df['lat'] - repd_osm_merge_df['latitude'])
max_lat_offset = max(repd_osm_merge_df['lat'] - repd_osm_merge_df['latitude'])
print(mean_lon_offset, max_lon_offset, mean_lat_offset, max_lat_offset)

-0.0005817911083584437 0.6631959126463869 0.0017672384138131346 1.3425169030472546


In [231]:
max_lon_offset_idx=(repd_osm_merge_df['lon'] - repd_osm_merge_df['longitude']).idxmax()

repd_osm_merge_df.iloc[max_lon_offset_idx]

Old Ref ID                                                        AA964
Ref ID                                                             1605
Record Last Updated (dd/mm/yyyy)                             10/06/2013
Operator (or Applicant)                    Lightsource Renewable Energy
Site Name                                             Blatchworthy Farm
Technology Type                                     Solar Photovoltaics
Storage Type                                                        NaN
Storage Co-location REPD Ref ID                                     NaN
Installed Capacity (MWelec)                                         1.5
CHP Enabled                                                         NaN
RO Banding (ROC/MWh)                                                NaN
FiT Tariff (p/kWh)                                                  NaN
CfD Capacity (MW)                                                   NaN
Turbine Capacity (MW)                                           

How many additional finds in OSM can we make, which don't have the REPD id already?
----
Check which rows in osm_df have a lat and lon within the max offsets for an repd row's lat and lon and note if the row has a 'tag_repd:id' that is still NaN, which means we could update osm_df with the corresponding repd row's 'Ref ID'.

In [170]:
osm_df[['lat', 'lon', 'tag_repd:id']].head()

,lat,lon,tag_repd:id
0,53.045913,-1.410764,NaN
1,57.148783,-6.211683,NaN
2,51.421184,0.824588,NaN
3,51.504658,-0.104954,NaN
4,51.464968,-0.073252,NaN


In [171]:
repd_df[['latitude', 'longitude', 'Ref ID']].head()

,latitude,longitude,Ref ID
0,51.387471,-3.406610,1
1,55.735807,-4.888574,2
2,53.716417,-1.281885,3
3,52.448628,0.725273,4
4,52.230195,-0.881341,5


In [172]:
for osm_index, osm_row in osm_df.iloc[0:200].iterrows():
    for repd_index, repd_row in repd_df.iterrows():
        if abs(repd_row['latitude'] - osm_row['lat']) < mean_lat_offset *2 and abs(repd_row['longitude'] - osm_row['lon']) < mean_lon_offset *2:
            osm_df.at[osm_index,'tag_repd:id'] = repd_row['Ref ID']

In [173]:
osm_df['tag_repd:id'].notnull().sum()

893

In [174]:

repd_df.loc[repd_df['Ref ID'] == 5]

,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,RO Banding (ROC/MWh),FiT Tariff (p/kWh),CfD Capacity (MW),Turbine Capacity (MW),No. of Turbines,Height of Turbines (m),Mounting Type for Solar,Development Status,Development Status (short),Address,County,Region,Country,Post Code,X-coordinate,Y-coordinate,Planning Authority,Planning Application Reference,Appeal Reference,Secretary of State Reference,Type of Secretary of State Intervention,Judicial Review,Offshore Wind Round,Planning Application Submitted,Planning Application Withdrawn,Planning Permission Refused,Appeal Lodged,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational,latitude,longitude
4,2047000,5,29/09/2005,Agrigen,Nunn Mills Road Biomass Plant,Biomass (dedicated),NaN,NaN,8.8,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Planning Permission Refused,Application Refused,"Nunn Mills Road\r, Northampton\r, Northamptons...",Northamptonshire,East Midlands,England,NaN,"476,500","259,700",Northamptonshire County Council,NaN,NaN,NaN,NaN,NaN,NaN,24/06/1997,NaN,01/02/1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.230195,-0.881341


In [175]:
# for osm_index, osm_row in osm_df.iterrows():
#     for repd_index, repd_row in repd_df.iterrows():
#         if abs(repd_row['latitude'] - osm_row['lat']) < max_lat_offset and abs(repd_row['longitude'] - osm_row['lon']) < max_lon_offset:
#             osm_df.set_value(osm_index,'tag_repd:id', repd_row['Ref ID'])

In [176]:
# df = osm_df.assign(a=1).merge(repd_df.assign(a=1), on='a')

# df = df[((df.latitude - df.lat).abs() < max_lat_offset) & 
#         ((df.longitude - df.lon).abs() < max_lon_offset)]

In [210]:
print(dict(osm_df['tag_repd:id']))

{0: nan, 1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: nan, 7: nan, 8: nan, 9: nan, 10: nan, 11: nan, 12: nan, 13: nan, 14: nan, 15: nan, 16: nan, 17: nan, 18: nan, 19: nan, 20: nan, 21: nan, 22: nan, 23: nan, 24: nan, 25: nan, 26: nan, 27: nan, 28: nan, 29: nan, 30: nan, 31: nan, 32: nan, 33: nan, 34: nan, 35: nan, 36: nan, 37: nan, 38: nan, 39: nan, 40: nan, 41: nan, 42: nan, 43: nan, 44: nan, 45: nan, 46: nan, 47: nan, 48: nan, 49: nan, 50: nan, 51: nan, 52: nan, 53: nan, 54: nan, 55: nan, 56: nan, 57: nan, 58: nan, 59: nan, 60: nan, 61: nan, 62: nan, 63: nan, 64: nan, 65: nan, 66: nan, 67: nan, 68: nan, 69: nan, 70: nan, 71: nan, 72: nan, 73: nan, 74: nan, 75: nan, 76: nan, 77: nan, 78: nan, 79: nan, 80: nan, 81: nan, 82: nan, 83: nan, 84: nan, 85: nan, 86: nan, 87: nan, 88: nan, 89: nan, 90: nan, 91: nan, 92: nan, 93: nan, 94: nan, 95: nan, 96: nan, 97: nan, 98: nan, 99: nan, 100: nan, 101: nan, 102: nan, 103: nan, 104: nan, 105: nan, 106: nan, 107: nan, 108: nan, 109: nan, 110: nan,

Find an example with a FiT entry
====


REPD 2143
----
Match on the capacity and post code

In [177]:
pd.set_option('display.max_columns', 500)

In [178]:
osm_df.loc[osm_df['tag_repd:id'] == 2143]

,objtype,id,user,timestamp,lat,lon,calc_area,calc_capacity,generator:solar:modules,location,orientation,plantref,tag_power,tag_repd:id,tag_start_date
120271,way,684035481,Russ,2019-05-03T21:41:29Z,51.935457,0.93541,230155.7,18800.0,NaN,NaN,NaN,way/684035481,plant,2143.0,NaN


In [179]:
repd_df.loc[repd_df['Ref ID'] == 2143]

,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,RO Banding (ROC/MWh),FiT Tariff (p/kWh),CfD Capacity (MW),Turbine Capacity (MW),No. of Turbines,Height of Turbines (m),Mounting Type for Solar,Development Status,Development Status (short),Address,County,Region,Country,Post Code,X-coordinate,Y-coordinate,Planning Authority,Planning Application Reference,Appeal Reference,Secretary of State Reference,Type of Secretary of State Intervention,Judicial Review,Offshore Wind Round,Planning Application Submitted,Planning Application Withdrawn,Planning Permission Refused,Appeal Lodged,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational,latitude,longitude
1731,C2404,2143,14/08/2015,NextEnergy Solar Fund,Boxted Airfield,Solar Photovoltaics,NaN,NaN,18.8,NaN,NaN,1.4,NaN,NaN,NaN,NaN,Ground,Operational,Operational,"Langham, Colchester",Essex,Eastern,England,CO4 5NW,"601,454","231,228",Colchester Borough Council,145101,NaN,NaN,NaN,NaN,NaN,16/06/2014,NaN,NaN,NaN,NaN,NaN,NaN,15/09/2014,NaN,NaN,NaN,NaN,26/11/2014,19/03/2015,51.943065,0.929579


In [180]:
fit_df.loc[(fit_df['Installed capacity'] > 17.8) & (fit_df['Installed capacity'] < 19.8) & (fit_df['Postcode'] == 'CO4 ')]

,Extension (Y/N),Postcode,Technology,Installed capacity,Declared net capacity,Application date,Commissioning date,MCS issue date,Export status,Tariff code,Tariff description,Installation type,Country,Local authority,Government office region,Constituency,Accreditation Route,MPAN Prefix,Community/school category,LLSOA code
73575,N,CO4,Photovoltaic,18.0,18.0,07/09/2015 00:00,27/07/2015 00:00,NaN,Export (Deemed),PV/10-50/06H-1,PV (>10Âª50kW)-2015/16 - Degression - H,Non Domestic (Commercial),England,Colchester,East of England,Colchester,MCS,10.0,Not Community Or School,E01021668


In [181]:
fit_df.loc[(fit_df['Declared net capacity'] > 17.8) & (fit_df['Declared net capacity'] < 19.8) & (fit_df['Postcode'] == 'CO4 ')]

,Extension (Y/N),Postcode,Technology,Installed capacity,Declared net capacity,Application date,Commissioning date,MCS issue date,Export status,Tariff code,Tariff description,Installation type,Country,Local authority,Government office region,Constituency,Accreditation Route,MPAN Prefix,Community/school category,LLSOA code
37504,N,CO4,Photovoltaic,22.6,18.7,08/06/2018 00:00,01/06/2018 00:00,07/06/2018 11:53,Export (Deemed),PV/10-50/09M-1,PV (>10-<=50kW)-2018/19 - M,Non Domestic (Commercial),England,Tendring,East of England,Harwich and North Essex,MCS,10.0,Not Community Or School,E01021971
73575,N,CO4,Photovoltaic,18.0,18.0,07/09/2015 00:00,27/07/2015 00:00,NaN,Export (Deemed),PV/10-50/06H-1,PV (>10Âª50kW)-2015/16 - Degression - H,Non Domestic (Commercial),England,Colchester,East of England,Colchester,MCS,10.0,Not Community Or School,E01021668


REPD 6132
-----

In [182]:
osm_df.loc[osm_df['tag_repd:id'] == 6132]

,objtype,id,user,timestamp,lat,lon,calc_area,calc_capacity,generator:solar:modules,location,orientation,plantref,tag_power,tag_repd:id,tag_start_date
126694,relation,9537182,Russ,2019-05-02T20:52:30Z,51.496088,-0.407339,9052.4,1700.0,NaN,NaN,NaN,relation/9537182,plant,6132.0,NaN


In [183]:
repd_df.loc[repd_df['Ref ID'] == 6132]

,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,RO Banding (ROC/MWh),FiT Tariff (p/kWh),CfD Capacity (MW),Turbine Capacity (MW),No. of Turbines,Height of Turbines (m),Mounting Type for Solar,Development Status,Development Status (short),Address,County,Region,Country,Post Code,X-coordinate,Y-coordinate,Planning Authority,Planning Application Reference,Appeal Reference,Secretary of State Reference,Type of Secretary of State Intervention,Judicial Review,Offshore Wind Round,Planning Application Submitted,Planning Application Withdrawn,Planning Permission Refused,Appeal Lodged,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational,latitude,longitude
4887,NaN,6132,31/05/2016,London Borough of Hounslow,Western International Market,Solar Photovoltaics,NaN,NaN,1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Roof,Operational,Operational,"Western International Market, Hayes Road, Sout...",Greater London,London,England,UB2 5XU,"510,606","178,655",Hounslow London Borough Council,P/2015/3360,NaN,NaN,NaN,NaN,NaN,28/07/2015,NaN,NaN,NaN,NaN,NaN,NaN,14/09/2015,NaN,NaN,NaN,NaN,19/04/2016,19/05/2016,51.496026,-0.408078


REPD 1
----
This example can't narrow down enough based on capacity and post code

In [197]:
repd_df.loc[repd_df['Ref ID'] == 1][['Post Code', 'Installed Capacity (MWelec)', 'latitude', 'longitude']]

,Post Code,Installed Capacity (MWelec),latitude,longitude
0,CF62 4ZW,35,51.387471,-3.40661


In [186]:
fit_df.loc[(fit_df['Installed capacity'] > 20) & (fit_df['Installed capacity'] < 50) & (fit_df['Postcode'] == 'CF62 ')]

,Extension (Y/N),Postcode,Technology,Installed capacity,Declared net capacity,Application date,Commissioning date,MCS issue date,Export status,Tariff code,Tariff description,Installation type,Country,Local authority,Government office region,Constituency,Accreditation Route,MPAN Prefix,Community/school category,LLSOA code
1132,N,CF62,Photovoltaic,22.50,22.50,25/03/2019 00:00,28/02/2019 00:00,01/03/2019 10:49,Export (Deemed),PV/10-50/09L-4,PV (>10-<=50kW)-2018/19 - L,Non Domestic (Commercial),Wales,The Vale of Glamorgan,Wales,Vale of Glamorgan,MCS,21.0,Not Community Or School,W01001102
119407,N,CF62,Photovoltaic,24.00,24.00,18/12/2015 00:00,27/08/2015 00:00,NaN,Export (Deemed),PV/10-50/06H-2,PV (>10Âª50kW)-2015/16 - Degression - H,Non Domestic (Commercial),Wales,The Vale of Glamorgan,Wales,Vale of Glamorgan,MCS,21.0,Not Community Or School,W01001095
480251,Y,CF62,Photovoltaic,49.82,49.82,27/04/2011 00:00,23/07/2012 00:00,NaN,No Export,PV/50-100/03 L,PV (>50Âª100kW)-2012/13 - Comp Review P1 - L,Non Domestic (Industrial),Wales,The Vale of Glamorgan,Wales,Vale of Glamorgan,MCS,NaN,Not Community Or School,W01001121


In [187]:
fit_df.loc[(fit_df['Declared net capacity'] > 20) & (fit_df['Declared net capacity'] < 50) & (fit_df['Postcode'] == 'CF62 ')]

,Extension (Y/N),Postcode,Technology,Installed capacity,Declared net capacity,Application date,Commissioning date,MCS issue date,Export status,Tariff code,Tariff description,Installation type,Country,Local authority,Government office region,Constituency,Accreditation Route,MPAN Prefix,Community/school category,LLSOA code
1132,N,CF62,Photovoltaic,22.50,22.50,25/03/2019 00:00,28/02/2019 00:00,01/03/2019 10:49,Export (Deemed),PV/10-50/09L-4,PV (>10-<=50kW)-2018/19 - L,Non Domestic (Commercial),Wales,The Vale of Glamorgan,Wales,Vale of Glamorgan,MCS,21.0,Not Community Or School,W01001102
119407,N,CF62,Photovoltaic,24.00,24.00,18/12/2015 00:00,27/08/2015 00:00,NaN,Export (Deemed),PV/10-50/06H-2,PV (>10Âª50kW)-2015/16 - Degression - H,Non Domestic (Commercial),Wales,The Vale of Glamorgan,Wales,Vale of Glamorgan,MCS,21.0,Not Community Or School,W01001095
258015,N,CF62,Photovoltaic,50.00,49.00,22/12/2014 00:00,19/12/2014 00:00,NaN,No Export,PV/10-50/05H-2,PV (>10Âª50kW)-2014/15 - Degression - H,Non Domestic (Commercial),Wales,The Vale of Glamorgan,Wales,Vale of Glamorgan,MCS,21.0,Not Community Or School,W01001119
480251,Y,CF62,Photovoltaic,49.82,49.82,27/04/2011 00:00,23/07/2012 00:00,NaN,No Export,PV/50-100/03 L,PV (>50Âª100kW)-2012/13 - Comp Review P1 - L,Non Domestic (Industrial),Wales,The Vale of Glamorgan,Wales,Vale of Glamorgan,MCS,NaN,Not Community Or School,W01001121


REPD 5207
--------

In [198]:
repd_df.loc[repd_df['Ref ID'] == 5207]

,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,RO Banding (ROC/MWh),FiT Tariff (p/kWh),CfD Capacity (MW),Turbine Capacity (MW),No. of Turbines,Height of Turbines (m),Mounting Type for Solar,Development Status,Development Status (short),Address,County,Region,Country,Post Code,X-coordinate,Y-coordinate,Planning Authority,Planning Application Reference,Appeal Reference,Secretary of State Reference,Type of Secretary of State Intervention,Judicial Review,Offshore Wind Round,Planning Application Submitted,Planning Application Withdrawn,Planning Permission Refused,Appeal Lodged,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational,latitude,longitude
4204,NaN,5207,17/06/2016,Canadian Solar,Land adjacent to Church Road,Solar Photovoltaics,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ground,Operational,Operational,"Land adjacent to Church Road and the M49, Seve...",Gloucestershire,South West,England,BS35 4NL,"354,510","184,911",South Gloucestershire Council,PT15/1586/F,NaN,NaN,NaN,NaN,NaN,13/04/2015,NaN,NaN,NaN,NaN,NaN,NaN,01/09/2015,NaN,NaN,NaN,01/09/2018,28/02/2015,31/03/2016,51.561233,-2.657613


In [199]:
osm_df.loc[osm_df['tag_repd:id'] == 5207]

,objtype,id,user,timestamp,lat,lon,calc_area,calc_capacity,generator:solar:modules,location,orientation,plantref,tag_power,tag_repd:id,tag_start_date
126934,relation,10304120,mcld,2019-11-16T14:03:13Z,51.558094,-2.655681,56859.6,3500.0,NaN,NaN,NaN,relation/10304120,plant,5207.0,NaN


REPD 1994
---

In [202]:
repd_df.loc[repd_df['Ref ID'] == 1994]

,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,RO Banding (ROC/MWh),FiT Tariff (p/kWh),CfD Capacity (MW),Turbine Capacity (MW),No. of Turbines,Height of Turbines (m),Mounting Type for Solar,Development Status,Development Status (short),Address,County,Region,Country,Post Code,X-coordinate,Y-coordinate,Planning Authority,Planning Application Reference,Appeal Reference,Secretary of State Reference,Type of Secretary of State Intervention,Judicial Review,Offshore Wind Round,Planning Application Submitted,Planning Application Withdrawn,Planning Permission Refused,Appeal Lodged,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational,latitude,longitude
1589,B1488,1994,26/08/2014,Private Developer,Hatch Penn Farm PV,Solar Photovoltaics,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ground,Operational,Operational,"Hatchpen Farm, Reed, Royston",Hertfordshire,Eastern,England,SG8 8AZ,"536,397","237,730",North Hertfordshire District Council,13/02128/1,NaN,NaN,NaN,NaN,NaN,24/11/2013,NaN,NaN,NaN,NaN,NaN,NaN,03/12/2013,NaN,NaN,NaN,NaN,NaN,08/03/2014,52.02123,-0.013603


In [203]:
osm_df.loc[osm_df['tag_repd:id'] == 1994]

,objtype,id,user,timestamp,lat,lon,calc_area,calc_capacity,generator:solar:modules,location,orientation,plantref,tag_power,tag_repd:id,tag_start_date
126938,relation,10305610,mcld,2019-11-16T21:30:41Z,52.019079,-0.011644,3573.2,1000.0,NaN,NaN,NaN,relation/10305610,plant,1994.0,NaN


REPD 5834
---

In [213]:
repd_df.loc[repd_df['Ref ID'] == 5834]

,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,RO Banding (ROC/MWh),FiT Tariff (p/kWh),CfD Capacity (MW),Turbine Capacity (MW),No. of Turbines,Height of Turbines (m),Mounting Type for Solar,Development Status,Development Status (short),Address,County,Region,Country,Post Code,X-coordinate,Y-coordinate,Planning Authority,Planning Application Reference,Appeal Reference,Secretary of State Reference,Type of Secretary of State Intervention,Judicial Review,Offshore Wind Round,Planning Application Submitted,Planning Application Withdrawn,Planning Permission Refused,Appeal Lodged,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational,latitude,longitude
4692,NaN,5834,04/12/2017,Greencoat Capital,Ebnal Lodge (resubmission),Solar Photovoltaics,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ground,Operational,Operational,"Land At, Rhosygadfa, Gobowen, Oswestry",Shropshire,West Midlands,England,NaN,"331,810","334,117",Shropshire Council,15/03975/FUL,NaN,NaN,NaN,NaN,NaN,23/09/2015,NaN,NaN,NaN,NaN,NaN,NaN,23/12/2015,NaN,NaN,NaN,23/12/2018,12/12/2016,15/02/2017,52.900149,-3.01518


In [212]:
osm_df.loc[osm_df['tag_repd:id'] == 5834]

,objtype,id,user,timestamp,lat,lon,calc_area,calc_capacity,generator:solar:modules,location,orientation,plantref,tag_power,tag_repd:id,tag_start_date
126930,relation,10298564,mcld,2019-11-14T22:35:13Z,52.89991,-3.015247,93095.9,5000.0,NaN,NaN,NaN,relation/10298564,plant,5834.0,NaN


Furthest apart linked in OSM/REPD: REPD 1605
-----

Lat offset: (51.146121 - 50.952354) x 110KM = **~21KM**

Lon offset: (-4.137665 - -3.474469) x 65KM = **~-43KM**

In [232]:
repd_df.loc[repd_df['Ref ID'] == 1605]

,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,RO Banding (ROC/MWh),FiT Tariff (p/kWh),CfD Capacity (MW),Turbine Capacity (MW),No. of Turbines,Height of Turbines (m),Mounting Type for Solar,Development Status,Development Status (short),Address,County,Region,Country,Post Code,X-coordinate,Y-coordinate,Planning Authority,Planning Application Reference,Appeal Reference,Secretary of State Reference,Type of Secretary of State Intervention,Judicial Review,Offshore Wind Round,Planning Application Submitted,Planning Application Withdrawn,Planning Permission Refused,Appeal Lodged,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational,latitude,longitude
1293,AA964,1605,10/06/2013,Lightsource Renewable Energy,Blatchworthy Farm,Solar Photovoltaics,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ground,Operational,Operational,"TIVERTON, DEVON",Devon,South West,England,EX16 9QH,"250,572","140,713",Mid Devon District Council,12/01381/MFUL,NaN,NaN,NaN,NaN,NaN,25/09/2012,NaN,NaN,NaN,NaN,NaN,NaN,22/03/2013,NaN,NaN,NaN,NaN,NaN,29/04/2013,51.146121,-4.137665


In [233]:
osm_df.loc[osm_df['tag_repd:id'] == 1605]

,objtype,id,user,timestamp,lat,lon,calc_area,calc_capacity,generator:solar:modules,location,orientation,plantref,tag_power,tag_repd:id,tag_start_date
120987,way,688883405,mcld,2019-11-16T18:42:25Z,50.952354,-3.474469,19338.6,1500.0,NaN,NaN,NaN,way/688883405,plant,1605.0,NaN


In [235]:
fit_df.loc[(fit_df['Installed capacity'] == 1.5) & (fit_df['Postcode'] == 'EX16 ')]

,Extension (Y/N),Postcode,Technology,Installed capacity,Declared net capacity,Application date,Commissioning date,MCS issue date,Export status,Tariff code,Tariff description,Installation type,Country,Local authority,Government office region,Constituency,Accreditation Route,MPAN Prefix,Community/school category,LLSOA code
4137,N,EX16,Photovoltaic,1.5,1.5,22/03/2019 00:00,20/03/2019 00:00,22/03/2019 18:44,Export (Deemed),PV-R/0-4/09H-4,PV (<=4kW (retrofit))-2018/19 - H,Domestic,England,Mid Devon,South West,Tiverton and Honiton,MCS,22.0,Not Community Or School,E01020051
75661,N,EX16,Photovoltaic,1.5,1.5,02/11/2016 00:00,06/10/2016 00:00,20/10/2016 15:17,Export (Deemed),PV-R/0-4/07H-3,PV (<=4kW (retrofit))-2016/17 - H,Domestic,England,Mid Devon,South West,Tiverton and Honiton,MCS,22.0,Not Community Or School,E01020047
86026,N,EX16,Photovoltaic,1.5,1.5,21/07/2015 00:00,21/04/2015 00:00,NaN,Export (Deemed),PV-R/0-4/06L-1,PV (<=4kW (retrofit))-2015/16 - Degression - L,Domestic,England,North Devon,South West,North Devon,MCS,22.0,Not Community Or School,E01020138
87433,N,EX16,Photovoltaic,1.5,1.5,20/03/2016 00:00,11/03/2016 00:00,11/03/2016 11:54,Export (Deemed),PV-N/0-4/06L-FR,PV (<=4kW (new build))-2015/16 - L - Post 2015...,Domestic,England,Mid Devon,South West,Tiverton and Honiton,MCS,22.0,Not Community Or School,E01020080
170127,N,EX16,Photovoltaic,1.5,1.5,26/10/2015 00:00,18/09/2015 00:00,NaN,Export (Deemed),PV-N/0-4/06H-2,PV (<=4kW (new build))-2015/16 - Degression - H,Domestic,England,Mid Devon,South West,Tiverton and Honiton,MCS,22.0,Not Community Or School,Unknown
600337,N,EX16,Photovoltaic,1.5,1.5,29/03/2012 00:00,27/03/2012 00:00,NaN,Export (Deemed),PV-R/0-4/02 R,PV (<=4kW (retrofit))-2011/12 - Comp Review P1...,Domestic,England,Mid Devon,South West,Tiverton and Honiton,MCS,22.0,Not Community Or School,E01020066
675617,N,EX16,Photovoltaic,1.5,1.5,27/10/2011 00:00,19/10/2011 00:00,NaN,Export (Deemed),PV-R/0-4/02,PV (<=4kW (retrofit))-2011/12,Domestic,England,Mid Devon,South West,Tiverton and Honiton,MCS,22.0,Not Community Or School,E01020066


Find a FiT entry in the other datasets
----

In [188]:
fit_df.iloc[1]

Extension (Y/N)                                        N
Postcode                                            LE4 
Technology                                  Photovoltaic
Installed capacity                                 18.63
Declared net capacity                              18.63
Application date                        04/01/2019 00:00
Commissioning date                      30/11/2018 00:00
MCS issue date                          12/12/2018 13:24
Export status                            Export (Deemed)
Tariff code                               PV/10-50/09L-3
Tariff description           PV (>10-<=50kW)-2018/19 - L
Installation type              Non Domestic (Commercial)
Country                                          England
Local authority                                Leicester
Government office region                   East Midlands
Constituency                              Leicester West
Accreditation Route                                  MCS
MPAN Prefix                    

In [189]:
repd_df.loc[repd_df['Region'] == 'East Midlands']['Address'].iloc[4]

'Corby Biomass Plant\r, Phoenix Parkway\r, Corby\r, Northamptonshire'

Ideas
====

1. Could it be useful to join dataframes based on location data and would this be easier with [GeoPandas](http://geopandas.org/mergingdata.html#spatial-joins)

Machine vision dataset
======

Dan Stowell notes: Ssimilar in to the OSM data, in the form of polygons, plus some metadata

In [18]:
import geopandas as gpd

machine_vision = "../data/as_received/machine-vision-descartes/Kruitwagen_Story_GB_fc.geojson"

machine_vision_df = gpd.read_file(machine_vision)

In [13]:
machine_vision_df.head()

,area,confidence,install_date,iso-3166-1,iso-3166-2,attribution,geometry
0,4415.687078,A,<2016-06,GB,GB-CON,University of Oxford & Descartes Labs Inc.,"POLYGON ((-4.89872 50.35443, -4.89870 50.35450..."
1,19465.080032,A,2017-01-25,GB,GB-CGN,University of Oxford & Descartes Labs Inc.,"POLYGON ((-4.13008 52.14918, -4.13007 52.14906..."
2,4010.630100,A,2017-01-25,GB,GB-CGN,University of Oxford & Descartes Labs Inc.,"POLYGON ((-4.13169 52.15059, -4.13166 52.15052..."
3,67758.309417,A,2016-07-19,GB,GB-SOM,University of Oxford & Descartes Labs Inc.,"POLYGON ((-2.86506 50.93777, -2.86465 50.93781..."
4,67758.309417,A,2016-07-19,GB,GB-SOM,University of Oxford & Descartes Labs Inc.,"POLYGON ((-2.87013 50.93961, -2.87012 50.93967..."


In [19]:
# x = []
# y = []
# for index, row in machine_vision_df.iterrows():
#     point = gpd.GeoSeries(machine_vision_df.centroid)
#     x = point.x
#     y = point.y
# machine_vision_df['x'] = x
# machine_vision_df['y'] = y

def getXY(pt):
    return (pt.x, pt.y)
centroidseries = machine_vision_df['geometry'].centroid
centroidlist = map(getXY, centroidseries)
machine_vision_df['x'], machine_vision_df['y'] = [list(t) for t in zip(*map(getXY, centroidseries))]
machine_vision_df.head()

,area,confidence,install_date,iso-3166-1,iso-3166-2,attribution,geometry,x,y
0,4415.687078,A,<2016-06,GB,GB-CON,University of Oxford & Descartes Labs Inc.,"POLYGON ((-4.89872 50.35443, -4.89870 50.35450...",-4.898207,50.354412
1,19465.080032,A,2017-01-25,GB,GB-CGN,University of Oxford & Descartes Labs Inc.,"POLYGON ((-4.13008 52.14918, -4.13007 52.14906...",-4.128699,52.148689
2,4010.630100,A,2017-01-25,GB,GB-CGN,University of Oxford & Descartes Labs Inc.,"POLYGON ((-4.13169 52.15059, -4.13166 52.15052...",-4.131029,52.150243
3,67758.309417,A,2016-07-19,GB,GB-SOM,University of Oxford & Descartes Labs Inc.,"POLYGON ((-2.86506 50.93777, -2.86465 50.93781...",-2.863473,50.937274
4,67758.309417,A,2016-07-19,GB,GB-SOM,University of Oxford & Descartes Labs Inc.,"POLYGON ((-2.87013 50.93961, -2.87012 50.93967...",-2.866699,50.939234


In [17]:
machine_vision_df['install_date']

0                 
1       2017-01-25
2       2017-01-25
3       2016-07-19
4       2016-07-19
           ...    
2216              
2217              
2218              
2219              
2220    2016-07-19
Name: install_date, Length: 2221, dtype: object

In [20]:
def remove_bad_date(dt_str):
    return(dt_str.replace("<2016-06", "").split(',')[0])
machine_vision_df['install_date'] = list(map(remove_bad_date, machine_vision_df['install_date']))
machine_vision_df.head()

,area,confidence,install_date,iso-3166-1,iso-3166-2,attribution,geometry,x,y
0,4415.687078,A,,GB,GB-CON,University of Oxford & Descartes Labs Inc.,"POLYGON ((-4.89872 50.35443, -4.89870 50.35450...",-4.898207,50.354412
1,19465.080032,A,2017-01-25,GB,GB-CGN,University of Oxford & Descartes Labs Inc.,"POLYGON ((-4.13008 52.14918, -4.13007 52.14906...",-4.128699,52.148689
2,4010.630100,A,2017-01-25,GB,GB-CGN,University of Oxford & Descartes Labs Inc.,"POLYGON ((-4.13169 52.15059, -4.13166 52.15052...",-4.131029,52.150243
3,67758.309417,A,2016-07-19,GB,GB-SOM,University of Oxford & Descartes Labs Inc.,"POLYGON ((-2.86506 50.93777, -2.86465 50.93781...",-2.863473,50.937274
4,67758.309417,A,2016-07-19,GB,GB-SOM,University of Oxford & Descartes Labs Inc.,"POLYGON ((-2.87013 50.93961, -2.87012 50.93967...",-2.866699,50.939234


In [21]:
machine_vision_df = machine_vision_df.drop(['geometry'], axis=1)

In [22]:
machine_vision_df.to_csv(index=False, path_or_buf="../data/raw/machine_vision.csv")

Exploring OSM data directly from pbf file
====


Reading pbf with python
-----

See:
1. [blog post](https://oslandia.com/en/2017/07/03/openstreetmap-data-analysis-how-to-parse-the-data-with-python/)
2. [stack example](https://stackoverflow.com/questions/45771809/how-to-extract-and-visualize-data-from-osm-file-in-python) from which the below code is copied

In [193]:
osm_file = "../data/open-street-maps/solarpv-osm-uk-data-20191117/gb-191117-solar-withreferenced.osm.pbf"

In [194]:
import osmium as osm

class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []

    def tag_inventory(self, elem, elem_type):
        for tag in elem.tags:
            self.osm_data.append([elem_type, 
                                   elem.id, 
                                   elem.version,
                                   elem.visible,
                                   pd.Timestamp(elem.timestamp),
                                   elem.uid,
                                   elem.user,
                                   elem.changeset,
                                   len(elem.tags),
                                   tag.k, 
                                   tag.v
                                 ])

    def node(self, n):
        self.tag_inventory(n, "node")

    def way(self, w):
        self.tag_inventory(w, "way")

    def relation(self, r):
        self.tag_inventory(r, "relation")


osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file(osm_file)

# transform the list into a pandas DataFrame
data_colnames = ['type', 'id', 'version', 'visible', 'ts', 'uid',
                 'user', 'chgset', 'ntags', 'tagkey', 'tagvalue']
df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
# df_osm = tag_genome.sort_values(by=['type', 'id', 'ts'])

In [195]:
df_osm.head()

,type,id,version,visible,ts,uid,user,chgset,ntags,tagkey,tagvalue
0,node,17342757,1,True,2006-10-01 21:54:21+00:00,1337,OJW,125916,2,natural,tree
1,node,17342757,1,True,2006-10-01 21:54:21+00:00,1337,OJW,125916,2,created_by,JOSM
2,node,17342760,1,True,2006-10-01 21:54:21+00:00,1337,OJW,125916,1,created_by,JOSM
3,node,17342780,1,True,2006-10-01 21:54:19+00:00,1337,OJW,125916,1,created_by,JOSM
4,node,29120995,1,True,2007-05-15 13:54:10+00:00,2274,Ben,52004,1,created_by,JOSM


In [196]:
osm_df.head()

,objtype,id,user,timestamp,lat,lon,calc_area,calc_capacity,generator:solar:modules,location,orientation,plantref,tag_power,tag_repd:id,tag_start_date
0,node,383729355,Dan023,2019-11-07T15:49:12Z,53.045913,-1.410764,0.0,NaN,NaN,roof,NaN,NaN,generator,NaN,NaN
1,node,432708661,don-vip,2011-06-03T16:19:07Z,57.148783,-6.211683,0.0,NaN,NaN,NaN,NaN,NaN,generator,NaN,NaN
2,node,754144142,Gregory Williams,2019-09-07T05:49:05Z,51.421184,0.824588,0.0,NaN,NaN,roof,NaN,NaN,generator,NaN,NaN
3,node,914501375,doublah,2019-08-09T16:21:10Z,51.504658,-0.104954,0.0,15.0,NaN,NaN,NaN,NaN,generator,NaN,2008
4,node,916163617,ehm1806,2012-12-14T07:16:28Z,51.464968,-0.073252,0.0,NaN,NaN,NaN,NaN,NaN,generator,NaN,NaN


In [236]:
osm_df['lat'] = osm_df['lat'].apply(lambda x: float(x))
osm_df['lon'] = osm_df['lon'].apply(lambda x: float(x))

In [237]:
osm_df.to_csv('../data/osm_modified.csv', sep=',', encoding='utf-8', index=False)

Writing pbf with new added data
--------

Possible workflow:
1. Read openstreetmaps solar pbf into python as above, but getting all the columns we want to use for matching in the pandas dataframe
2. Add new columns or update data using the other data sources as I have done with REPD tags based on coordinate comparison in this notebook
3. [Convert the df to JSON](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html)
4. [Use Osmium to convert back to pbf](https://oslandia.com/en/2017/07/03/openstreetmap-data-analysis-how-to-parse-the-data-with-python/)
